In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_precision_recall

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

In [2]:
pp_train = pd.read_csv("pp_train.csv")
#pp_train.head()

In [3]:
pp_train["titleType"].unique()

array(['tvEpisode', 'videoGame', 'movie', 'tvSeries', 'video',
       'tvMiniSeries', 'short', 'tvMovie', 'tvSpecial', 'tvShort'],
      dtype=object)

In [4]:
pp_train.columns

Index(['originalTitle', 'rating', 'startYear', 'endYear', 'runtimeMinutes',
       'awardWins', 'numVotes', 'totalImages', 'totalVideos', 'totalCredits',
       'criticReviewsTotal', 'titleType', 'awardNominationsExcludeWins',
       'canHaveEpisodes', 'isAdult', 'numRegions', 'userReviewsTotal',
       'countryOfOrigin', 'is_Sport', 'is_Family', 'is_Talk-Show', 'is_Action',
       'is_Biography', 'is_Adventure', 'is_Short', 'is_Adult', 'is_Drama',
       'is_Musical', 'is_Documentary', 'is_Reality-TV', 'is_Thriller',
       'is_Comedy', 'is_Film-Noir', 'is_History', 'is_War', 'is_News',
       'is_Fantasy', 'is_Horror', 'is_Animation', 'is_Romance', 'is_Music',
       'is_Crime', 'is_Mystery', 'is_Western', 'is_Sci-Fi', 'is_Game-Show',
       'fill_runtimeMinutes'],
      dtype='object')

### creo copia di pp_train su cui lavorare

In [5]:
pp_class = pp_train.copy()
type(pp_class)

pandas.core.frame.DataFrame

## pp_class
### codifica variabile target

In [6]:

y = np.array(pp_class['titleType'])  # Il target originale

# Creiamo un dizionario che associa ogni categoria a un numero
y_map = {title: idx for idx, title in enumerate(pp_class['titleType'].unique())}

#aggiungo una nuova colonna
pp_class['titleType_encoded'] = pp_class['titleType'].map(y_map)

print(y_map)  # Per vedere la mappatura delle categorie

{'tvEpisode': 0, 'videoGame': 1, 'movie': 2, 'tvSeries': 3, 'video': 4, 'tvMiniSeries': 5, 'short': 6, 'tvMovie': 7, 'tvSpecial': 8, 'tvShort': 9}


In [7]:
# print(pp_class[['titleType', 'titleType_encoded']].head())

## feature engeneering -> su pp_class

##### info variabili

In [8]:
#studio variabili (non ho messo le 28 is_genere)
pp_class[['originalTitle', 'rating', 'startYear', 'endYear', 'runtimeMinutes', 'awardWins', 'numVotes', 'totalImages', 'totalVideos', 'totalCredits', 'criticReviewsTotal', 'titleType', 'awardNominationsExcludeWins', 'canHaveEpisodes', 'isAdult', 'numRegions', 'userReviewsTotal','countryOfOrigin', 
       'fill_runtimeMinutes', 'titleType_encoded']].head(3)

,originalTitle,rating,startYear,endYear,runtimeMinutes,awardWins,numVotes,totalImages,totalVideos,totalCredits,criticReviewsTotal,titleType,awardNominationsExcludeWins,canHaveEpisodes,isAdult,numRegions,userReviewsTotal,countryOfOrigin,fill_runtimeMinutes,titleType_encoded
0,Neogenic Nightmare Chapter 3: Hydro-Man,8,1995,NaN,NaN,0.0,779,1,0,21,1,tvEpisode,0,False,0,1,0,['US'],40.0,0
1,Looping,6,1982,NaN,NaN,0.0,11,1,0,1,0,videoGame,0,False,0,1,0,['XWG'],28.0,1
2,Idealnaya para,6,1992,NaN,NaN,0.0,38,1,0,24,0,movie,0,False,0,1,1,['RU'],90.0,2


NO -> 'originalTitle' -> togliere
    'rating' -> valori da 0 a 10 (intt64)
    'startYear' -> continua
    'endYear' -> continua ma con Nan
 NO ->  'runtimeMinutes'
     'awardWins' -> continua -> TANTI ZERI -> forse dividere in 0 e diverso da 0
    'numVotes'-> continua ()
    'totalImages'> continua ()
    'totalVideos' -> continua -> TANTI ZERI ->forse dividere in 0 e diverso da 0
    'totalCredits' -> continua ()
    'criticReviewsTotal' -> continua
NO ->'titleType' -> messa encoded
    'awardNominationsExcludeWins' -> continua -> TANTI ZERI ->forse dividere in 0 e diverso da 0
CAMBIA -'canHaveEpisodes' -> booleana True/False
    'isAdult' -> 0/1
    'numRegions'-> continua ()
    'userReviewsTotal'-> continua ()
? -> 'countryOfOrigin' -> stringa con liste di stringhe
    'is_Documentary', 'is_History', 'is_Adventure','is_Thriller', 'is_Game-Show', 'is_Comedy', 'is_Sci-Fi', 'is_Romance', 'is_Biography',   
            'is_Musical', 'is_Western', 'is_Music', 'is_Film-Noir', 'is_Adult', 'is_Reality-TV', 'is_News', 'is_Action', 'is_Crime', 'is_Short', 'is_Fantasy', 'is_Family', 'is_Mystery', 'is_Talk-Show', 'is_Drama', 'is_Sport', 'is_War', 'is_Horror', 'is_Animation',
        28 generi -> abbiamo fatto onehot encoding -> a un titolo pox corrispondere + generi  
    'fill_runtimeMinutes' -> continua

#### feature engeneriing

In [9]:
#variabili da eliminare
column2drop = ['originalTitle', 'runtimeMinutes', 'titleType']
pp_class.drop(column2drop, axis=1, inplace=True)

#variab categoriale da convertire in 0/1
pp_class['canHaveEpisodes'] = pp_class['canHaveEpisodes'].astype(int)

#binarizzazione (0 / >0) per le variabili con tanti zeri (consigliato per NB)
    # Alternativa: Se vuoi mantenere i valori numerici, puoi discretizzare in fasce (es. 0, 1-5, >5).
pp_class['awardWins_bin'] = (pp_class['awardWins'] > 0).astype(int)
pp_class['totalVideos_bin'] = (pp_class['totalVideos'] > 0).astype(int)
pp_class['awardNominations_bin'] = (pp_class['awardNominationsExcludeWins'] > 0).astype(int)

# Riempi i valori mancanti di 'endYear' con quelli di 'startYear'
pp_class['endYear'].fillna(pp_class['startYear'], inplace=True)


In [10]:
pp_class.head()

,rating,startYear,endYear,awardWins,numVotes,totalImages,totalVideos,totalCredits,criticReviewsTotal,awardNominationsExcludeWins,...,is_Crime,is_Mystery,is_Western,is_Sci-Fi,is_Game-Show,fill_runtimeMinutes,titleType_encoded,awardWins_bin,totalVideos_bin,awardNominations_bin
0,8,1995,1995.0,0.0,779,1,0,21,1,0,...,0,0,0,0,0,40.0,0,0,0,0
1,6,1982,1982.0,0.0,11,1,0,1,0,0,...,0,0,0,0,0,28.0,1,0,0,0
2,6,1992,1992.0,0.0,38,1,0,24,0,0,...,0,0,0,0,0,90.0,2,0,0,0
3,6,2021,2021.0,0.0,15,22,0,56,0,0,...,0,0,0,0,0,31.0,3,0,0,0
4,8,2020,2020.0,1.0,15718,347,2,47,4,1,...,0,0,0,0,0,80.0,3,1,1,1


### gestione country of origin 

In [11]:
#la ritrasformo in lista di stringhe

import ast
pp_class['countryOfOrigin'] = pp_class['countryOfOrigin'].apply(ast.literal_eval)

#print(pp_class['countryOfOrigin'].apply(lambda x: [type(country) for country in x]))
val = pp_class.loc[8758,"countryOfOrigin"]
type(val)


list

### variabili utili

In [12]:
# Flatten all lists and get unique countries
all_countries = set(country for sublist in pp_class['countryOfOrigin'] if isinstance(sublist, list) for country in sublist)
print(all_countries)

{'VE', 'AU', 'TW', 'IT', 'BG', 'UZ', 'ID', 'GA', 'HU', 'MQ', 'LS', 'NE', 'ET', 'XWG', 'AR', 'SY', 'DZ', 'SV', 'GB', 'RO', 'DE', 'GT', 'TN', 'UA', 'EC', 'SI', 'TG', 'CZ', 'AL', 'JP', 'MK', 'HR', 'CSXX', 'HT', 'CR', 'LT', 'KZ', 'PY', 'KP', 'AE', 'GM', 'EH', 'BD', 'ML', 'KG', 'BB', 'PK', 'CY', 'SG', 'LK', 'ES', 'JO', 'CL', 'EE', 'VA', 'GW', 'CA', 'AF', 'DDDE', 'NP', 'PR', 'AO', 'BH', 'RW', 'NA', 'MO', 'NZ', 'PL', 'KY', 'TR', 'DK', 'SA', 'AM', 'GP', 'SM', 'IN', 'EG', 'NG', 'GR', 'BY', 'LC', 'PT', 'QA', 'HK', 'AZ', 'IL', 'JM', 'PH', 'SE', 'VC', 'BF', 'TT', 'KE', 'LY', 'LI', 'CO', 'FI', 'IS', 'BE', 'YUCS', 'XKV', 'IE', 'ME', 'MY', 'NO', 'BO', 'XYU', 'CM', 'CU', 'KH', 'LU', 'GD', 'ZW', 'BR', 'MM', 'PE', 'FR', 'RU', 'CD', 'SR', 'MX', 'US', 'UY', 'VN', 'AD', 'GL', 'AT', 'GN', 'SUHH', 'MC', 'IQ', 'KR', 'IR', 'BA', 'ZA', 'TO', 'CH', 'SK', 'RS', 'CN', 'GY', 'ZM', 'MA', 'SN', 'KW', 'GE', 'TH', 'LB', 'CSHH', 'DO', 'LV', 'MT', 'NL'}


In [13]:
# Dizionario completo di codici storici di paesi
HISTORIC_CODES = {
    # Europa
    'XYU': {'name': 'Yugoslavia', 'continent': 'Europe'},
    'YUCS': {'name': 'Yugoslavia (Socialist Federal Republic)', 'continent': 'Europe'},
    'YUG': {'name': 'Yugoslavia', 'continent': 'Europe'},
    'SUHH': {'name': 'Soviet Union', 'continent': 'Europe'},
    'SUN': {'name': 'Soviet Union', 'continent': 'Europe'},
    'SU': {'name': 'Soviet Union', 'continent': 'Europe'},
    'USSR': {'name': 'Union of Soviet Socialist Republics', 'continent': 'Europe'},
    'DDDE': {'name': 'East Germany', 'continent': 'Europe'},
    'DDR': {'name': 'German Democratic Republic', 'continent': 'Europe'},
    'XWG': {'name': 'East Germany', 'continent': 'Europe'},
    'BRD': {'name': 'Federal Republic of Germany (West Germany)', 'continent': 'Europe'},
    'FRGG': {'name': 'West Germany', 'continent': 'Europe'},
    'CSXX': {'name': 'Czechoslovakia', 'continent': 'Europe'},
    'CSHH': {'name': 'Czechoslovakia', 'continent': 'Europe'},
    'CSK': {'name': 'Czechoslovakia', 'continent': 'Europe'},
    'TCH': {'name': 'Czechoslovakia', 'continent': 'Europe'},
    'XCZ': {'name': 'Czechoslovakia (1945-1992)', 'continent': 'Europe'},
    'SCG': {'name': 'Serbia and Montenegro', 'continent': 'Europe'},
    'SCGN': {'name': 'Serbia and Montenegro', 'continent': 'Europe'},
    'XKV': {'name': 'Kosovo', 'continent': 'Europe'},
    'XPL': {'name': 'Poland (1945-1989)', 'continent': 'Europe'},
    'XHU': {'name': 'Hungary (1945-1989)', 'continent': 'Europe'},
    'XEU': {'name': 'European Union', 'continent': 'Europe'},
    'EUE': {'name': 'European Union', 'continent': 'Europe'},
    'XCI': {'name': 'Channel Islands', 'continent': 'Europe'},
    'XES': {'name': 'Estonia (pre-1991)', 'continent': 'Europe'},
    'XLV': {'name': 'Latvia (pre-1991)', 'continent': 'Europe'},
    'XLT': {'name': 'Lithuania (pre-1991)', 'continent': 'Europe'},
    'XAZ': {'name': 'Azores', 'continent': 'Europe'},
    'XFR': {'name': 'France (Historical)', 'continent': 'Europe'},
    'FXX': {'name': 'Metropolitan France', 'continent': 'Europe'},
    'AHU': {'name': 'Austria-Hungary', 'continent': 'Europe'},
    'PRU': {'name': 'Prussia', 'continent': 'Europe'},
    'BAV': {'name': 'Bavaria', 'continent': 'Europe'},
    'KPS': {'name': 'Kingdom of the Two Sicilies', 'continent': 'Europe'},
    'VEN': {'name': 'Republic of Venice', 'continent': 'Europe'},
    'XOH': {'name': 'Ottoman Hungary', 'continent': 'Europe'},
    'XCR': {'name': 'Crimean Khanate', 'continent': 'Europe'},
    'MON': {'name': 'Montenegro (pre-independence)', 'continent': 'Europe'},
    'VA': {'name': 'Vatican City State', 'continent': 'Europe'},
    'SRB': {'name': 'Kingdom of Serbia', 'continent': 'Europe'},
    'PDM': {'name': 'Piedmont-Sardinia', 'continent': 'Europe'},
    'XPP': {'name': 'Papal States', 'continent': 'Europe'},
    'XMO': {'name': 'Monaco (Historical)', 'continent': 'Europe'},
    'XSM': {'name': 'San Marino (Historical)', 'continent': 'Europe'},
    'XLI': {'name': 'Liechtenstein (Historical)', 'continent': 'Europe'},
    
    # Asia
    'ANT': {'name': 'Netherlands Antilles', 'continent': 'North_America'},
    'ANHH': {'name': 'Netherlands Antilles', 'continent': 'North_America'},
    'XAN': {'name': 'Netherlands Antilles', 'continent': 'North_America'},
    'BUR': {'name': 'Burma (now Myanmar)', 'continent': 'Asia'},
    'TPE': {'name': 'Chinese Taipei (Taiwan)', 'continent': 'Asia'},
    'XIR': {'name': 'Persia (now Iran)', 'continent': 'Asia'},
    'TMP': {'name': 'East Timor (pre-independence)', 'continent': 'Asia'},
    'TLS': {'name': 'East Timor', 'continent': 'Asia'},
    'XTI': {'name': 'East Timor (Portuguese Timor)', 'continent': 'Asia'},
    'YMD': {'name': 'South Yemen (People\'s Democratic Republic)', 'continent': 'Asia'},
    'YMN': {'name': 'North Yemen (Yemen Arab Republic)', 'continent': 'Asia'},
    'XHK': {'name': 'Hong Kong (pre-1997)', 'continent': 'Asia'},
    'HKJ': {'name': 'Hashemite Kingdom of Jordan', 'continent': 'Asia'},
    'XNA': {'name': 'Neutral Zone (Saudi Arabia-Iraq)', 'continent': 'Asia'},
    'XEA': {'name': 'East Asia (historical region)', 'continent': 'Asia'},
    'XSE': {'name': 'Southeast Asia (historical region)', 'continent': 'Asia'},
    'XME': {'name': 'Middle East (region)', 'continent': 'Asia'},
    'XIO': {'name': 'British Indian Ocean Territory', 'continent': 'Asia'},
    'XSI': {'name': 'Sikkim (now part of India)', 'continent': 'Asia'},
    #'XEG': {'name': 'United Arab Republic (Egypt & Syria)', 'continent': 'Africa/Asia'},
    'XMB': {'name': 'Manchukuo (Japanese puppet state in Manchuria)', 'continent': 'Asia'},
    'KOR': {'name': 'Korea (pre-division)', 'continent': 'Asia'},
    'XSM': {'name': 'Siam (now Thailand)', 'continent': 'Asia'},
    'XCY': {'name': 'Ceylon (now Sri Lanka)', 'continent': 'Asia'},
    'SAA': {'name': 'Sarawak (pre-Malaysia)', 'continent': 'Asia'},
    'SBH': {'name': 'North Borneo (now Sabah)', 'continent': 'Asia'},
    'XPH': {'name': 'Philippines (American period)', 'continent': 'Asia'},
    'XIN': {'name': 'British India', 'continent': 'Asia'},
    'GBG': {'name': 'Great Burhan Regime (Afghanistan)', 'continent': 'Asia'},
    'XPL': {'name': 'Palestine (British Mandate)', 'continent': 'Asia'},
    'XFI': {'name': 'French Indochina', 'continent': 'Asia'},
    'XDP': {'name': 'Portuguese India', 'continent': 'Asia'},
    'XMO': {'name': 'Macao (pre-1999)', 'continent': 'Asia'},
    'XQG': {'name': 'Qing Dynasty China', 'continent': 'Asia'},
    
    # Africa
    'RHO': {'name': 'Rhodesia (now Zimbabwe)', 'continent': 'Africa'},
    'XRH': {'name': 'Southern Rhodesia', 'continent': 'Africa'},
    'ZAR': {'name': 'Zaire (now Democratic Republic of the Congo)', 'continent': 'Africa'},
    'XAF': {'name': 'Afars and Issas (now Djibouti)', 'continent': 'Africa'},
    'XDY': {'name': 'Dahomey (now Benin)', 'continent': 'Africa'},
    'XUG': {'name': 'Uganda (pre-1962)', 'continent': 'Africa'},
    'XET': {'name': 'Ethiopia (pre-1993)', 'continent': 'Africa'},
    'XSU': {'name': 'Sudan (pre-2011)', 'continent': 'Africa'},
    'USR': {'name': 'Upper Volta (now Burkina Faso)', 'continent': 'Africa'},
    'XVO': {'name': 'Upper Volta', 'continent': 'Africa'},
    'XBF': {'name': 'Burkina Faso (1984-present)', 'continent': 'Africa'},
    'XLI': {'name': 'Libya (pre-2011)', 'continent': 'Africa'},
    'XTC': {'name': 'Tanganyika (now part of Tanzania)', 'continent': 'Africa'},
    'XZR': {'name': 'Zanzibar (now part of Tanzania)', 'continent': 'Africa'},
    'XTZ': {'name': 'Tanzania (1964-present)', 'continent': 'Africa'},
    'XGH': {'name': 'Gold Coast (now Ghana)', 'continent': 'Africa'},
    'XSL': {'name': 'Sierra Leone (pre-1961)', 'continent': 'Africa'},
    'XMA': {'name': 'Madeira', 'continent': 'Africa'},
    'XTA': {'name': 'Tangier International Zone', 'continent': 'Africa'},
    'XCV': {'name': 'Cape Verde (before 2013)', 'continent': 'Africa'},
    'XBI': {'name': 'British Somaliland', 'continent': 'Africa'},
    'XIF': {'name': 'Italian Somaliland', 'continent': 'Africa'},
    'XAL': {'name': 'Algérie française (French Algeria)', 'continent': 'Africa'},
    'XNY': {'name': 'Nyasaland (now Malawi)', 'continent': 'Africa'},
    'XBE': {'name': 'Belgian Congo', 'continent': 'Africa'},
    'XBA': {'name': 'Basutoland (now Lesotho)', 'continent': 'Africa'},
    'XSW': {'name': 'South West Africa (now Namibia)', 'continent': 'Africa'},
    'XZB': {'name': 'Zambezi (colonial term)', 'continent': 'Africa'},
    'XAO': {'name': 'Angola (Portuguese colony)', 'continent': 'Africa'},
    'EH': {'name': 'Western Sahara (Sahara Occidental)', 'continent': 'Africa'},
    'XEN': {'name': 'Emirates of North Africa', 'continent': 'Africa'},
    'XTR': {'name': 'Tripolitania (now part of Libya)', 'continent': 'Africa'},
    'XCY': {'name': 'Cyrenaica (now part of Libya)', 'continent': 'Africa'},
    'XAB': {'name': 'Abyssinia (historical Ethiopia)', 'continent': 'Africa'},
    'XBB': {'name': 'Bechuanaland (now Botswana)', 'continent': 'Africa'},
    'XMA': {'name': 'Madagascar (French colony)', 'continent': 'Africa'},
    'XMO': {'name': 'Morocco (French/Spanish protectorate)', 'continent': 'Africa'},
    'XTU': {'name': 'Tunisia (French protectorate)', 'continent': 'Africa'},
    
    # Americas
    'XUS': {'name': 'United States (Historical)', 'continent': 'North_America'},
    'XCA': {'name': 'Canada (Historical)', 'continent': 'North_America'},
    'XPU': {'name': 'Panama Canal Zone', 'continent': 'North_America'},
    'XMX': {'name': 'Mexico (Historical)', 'continent': 'North_America'},
    'XBR': {'name': 'Brazil (Historical)', 'continent': 'South_America'},
    'XAR': {'name': 'Argentina (Historical)', 'continent': 'South_America'},
    'XCB': {'name': 'Caribbean Islands (collective)', 'continent': 'North_America'},
    'XCL': {'name': 'Chile (pre-1990)', 'continent': 'South_America'},
    'XCO': {'name': 'Colombia (pre-1991)', 'continent': 'South_America'},
    'XCU': {'name': 'Cuba (pre-1959)', 'continent': 'North_America'},
    'XEN': {'name': 'English Caribbean Islands', 'continent': 'North_America'},
    'XFR': {'name': 'French Caribbean Islands', 'continent': 'North_America'},
    'XGL': {'name': 'Greenland (pre-1979)', 'continent': 'North_America'},
    'XGY': {'name': 'Guyana (British Guiana)', 'continent': 'South_America'},
    'XSR': {'name': 'Suriname (Dutch Guiana)', 'continent': 'South_America'},
    'XGF': {'name': 'French Guiana (Historical)', 'continent': 'South_America'},
    'XPI': {'name': 'Saint-Pierre and Miquelon', 'continent': 'North_America'},
    'USC': {'name': 'Confederate States of America', 'continent': 'North_America'},
    'XTX': {'name': 'Republic of Texas', 'continent': 'North_America'},
    'XHT': {'name': 'Saint-Domingue (now Haiti)', 'continent': 'North_America'},
    'XBH': {'name': 'British Honduras (now Belize)', 'continent': 'North_America'},
    'XBW': {'name': 'British West Indies', 'continent': 'North_America'},
    'XUN': {'name': 'United Provinces of Central America', 'continent': 'North_America'},
    'XGP': {'name': 'Gran Colombia', 'continent': 'South_America'},
    'XPR': {'name': 'Peru-Bolivian Confederation', 'continent': 'South_America'},
    'XPN': {'name': 'Panama (as part of Colombia)', 'continent': 'North_America'},
    'XRI': {'name': 'Río de la Plata (Viceroyalty)', 'continent': 'South_America'},
    'XLN': {'name': 'New Granada (Viceroyalty)', 'continent': 'South_America'},
    'XNS': {'name': 'New Spain (Viceroyalty)', 'continent': 'North_America'},
    'XBE': {'name': 'British Empire in America', 'continent': 'North_America'},
    'XDW': {'name': 'Danish West Indies (now US Virgin Islands)', 'continent': 'North_America'},
    
    # Oceania
    'XAU': {'name': 'Australia (pre-Federation)', 'continent': 'Oceania'},
    'XNZ': {'name': 'New Zealand (pre-1907)', 'continent': 'Oceania'},
    'PCI': {'name': 'Pacific Islands Trust Territory', 'continent': 'Oceania'},
    'XPI': {'name': 'Pacific Islands (US Trust Territory)', 'continent': 'Oceania'},
    'XFJ': {'name': 'Fiji (pre-1970)', 'continent': 'Oceania'},
    'XPN': {'name': 'Papua New Guinea (pre-1975)', 'continent': 'Oceania'},
    'XPW': {'name': 'Palau (pre-1994)', 'continent': 'Oceania'},
    'XMH': {'name': 'Marshall Islands (pre-1986)', 'continent': 'Oceania'},
    'XFM': {'name': 'Federated States of Micronesia (pre-1986)', 'continent': 'Oceania'},
    'XGE': {'name': 'Gilbert Islands (now part of Kiribati)', 'continent': 'Oceania'},
    'XEL': {'name': 'Ellice Islands (now Tuvalu)', 'continent': 'Oceania'},
    'XNH': {'name': 'New Hebrides (now Vanuatu)', 'continent': 'Oceania'},
    'PCT': {'name': 'Pacific Islands Trust Territory', 'continent': 'Oceania'},
    'XWS': {'name': 'Western Samoa (now Samoa)', 'continent': 'Oceania'},
    'XNA': {'name': 'Netherlands New Guinea', 'continent': 'Oceania'},
    'XHS': {'name': 'Hawaii (pre-US statehood)', 'continent': 'Oceania'},
    'XBP': {'name': 'British Polynesia', 'continent': 'Oceania'},
    'XFP': {'name': 'French Polynesia (historical)', 'continent': 'Oceania'},
    'XNC': {'name': 'New Caledonia (historical)', 'continent': 'Oceania'},
    'XSI': {'name': 'Solomon Islands (British protectorate)', 'continent': 'Oceania'},
    
}

###  Ottiene il continente per una singola sigla di paese

In [14]:
import pycountry
import pycountry_convert as pc


def get_continent(country_code):
    """Ottiene il continente per una singola sigla di paese"""
    # Controlla prima nel dizionario storico
    if country_code in HISTORIC_CODES:
        return HISTORIC_CODES[country_code]['continent']
    
    try:
        # Per codici a 2 lettere (alpha-2)
        if len(country_code) == 2:
            country_continent_code = pc.country_alpha2_to_continent_code(country_code)
            continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
            return continent_name
            
        # Per codici a 3 lettere (alpha-3)
        elif len(country_code) == 3:
            # Converti da alpha-3 a alpha-2
            country = pycountry.countries.get(alpha_3=country_code)
            if country:
                country_alpha2 = country.alpha_2
                country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
                continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
                return continent_name
            
        # Controlla nei paesi storici di pycountry
        historic = next((c for c in pycountry.historic_countries if c.alpha_3 == country_code), None)
        if historic:
            return map_historic_to_continent(historic.name)
                
        return None
    except (KeyError, ValueError, AttributeError):
        return None

def get_continents_for_list(country_codes):
    """Gestisce una lista di sigle di paesi e restituisce un dizionario con i risultati"""
    results = {}
    
    # Se l'input è una singola stringa, convertila in lista
    if isinstance(country_codes, str):
        country_codes = [country_codes]
    
    # Elabora ogni sigla nella lista
    for code in country_codes:
        results[code] = get_continent(code)
    
    return results

def map_historic_to_continent(country_name):
    """Mappa manuale di alcuni paesi storici ai loro continenti"""
    mappings = {
        'USSR': 'Europe',
        'YUGOSLAVIA': 'Europe',
        'CZECHOSLOVAKIA': 'Europe',
        # Aggiungi altri paesi storici secondo necessità
    }
    
    for key, value in mappings.items():
        if key in country_name.upper():
            return value
    
    return None

# utilizzo con lista
all_countries = set(country for sublist in pp_class['countryOfOrigin'] if isinstance(sublist, list) for country in sublist)
continent_mapping = get_continents_for_list(all_countries)
continent_mapping


{'VE': 'South America',
 'AU': 'Oceania',
 'TW': 'Asia',
 'IT': 'Europe',
 'BG': 'Europe',
 'UZ': 'Asia',
 'ID': 'Asia',
 'GA': 'Africa',
 'HU': 'Europe',
 'MQ': 'North America',
 'LS': 'Africa',
 'NE': 'Africa',
 'ET': 'Africa',
 'XWG': 'Europe',
 'AR': 'South America',
 'SY': 'Asia',
 'DZ': 'Africa',
 'SV': 'North America',
 'GB': 'Europe',
 'RO': 'Europe',
 'DE': 'Europe',
 'GT': 'North America',
 'TN': 'Africa',
 'UA': 'Europe',
 'EC': 'South America',
 'SI': 'Europe',
 'TG': 'Africa',
 'CZ': 'Europe',
 'AL': 'Europe',
 'JP': 'Asia',
 'MK': 'Europe',
 'HR': 'Europe',
 'CSXX': 'Europe',
 'HT': 'North America',
 'CR': 'North America',
 'LT': 'Europe',
 'KZ': 'Asia',
 'PY': 'South America',
 'KP': 'Asia',
 'AE': 'Asia',
 'GM': 'Africa',
 'EH': 'Africa',
 'BD': 'Asia',
 'ML': 'Africa',
 'KG': 'Asia',
 'BB': 'North America',
 'PK': 'Asia',
 'CY': 'Asia',
 'SG': 'Asia',
 'LK': 'Asia',
 'ES': 'Europe',
 'JO': 'Asia',
 'CL': 'South America',
 'EE': 'Europe',
 'VA': 'Europe',
 'GW': 'Africa

In [15]:
import pandas as pd

def replace_country_with_continent(df, column_name, continent_mapping):
    """
    Sostituisce i codici dei paesi nella colonna di un DataFrame con il loro continente corrispondente.
    
    :param df: DataFrame contenente la colonna da modificare
    :param column_name: Nome della colonna con le liste di codici paese
    :param continent_mapping: Dizionario {codice_paese: continente}
    :return: DataFrame con la colonna modificata
    """
    df[column_name] = df[column_name].apply(
        lambda country_list: list(set(continent_mapping.get(country, 'Unknown') for country in country_list))
    )
    return df


# Applicazione della funzione
df = replace_country_with_continent(pp_class, 'countryOfOrigin', get_continents_for_list(all_countries))

# Stampa del risultato
print(pp_class["countryOfOrigin"])

print(pp_class.loc[8758,"countryOfOrigin"])


0        [North America]
1               [Europe]
2               [Europe]
3        [North America]
4                 [Asia]
              ...       
16426           [Europe]
16427             [Asia]
16428             [Asia]
16429           [Europe]
16430             [Asia]
Name: countryOfOrigin, Length: 16431, dtype: object
['Europe', 'North America']


#### creazione attributi is_from_X per one hot encoding come per generi (ci sono multivalori)

In [16]:
# Flatten all lists and get unique genres
all_continents = set(cont for sublist in pp_class['countryOfOrigin'] if isinstance(sublist, list) for cont in sublist)
print(all_continents)

# Create a one-hot encoded column for each continent
for genre in all_continents:
    pp_class[f"is_from_{genre}"] = pp_class['countryOfOrigin'].apply(lambda x: 1 if genre in x else 0)



#elimino colonna country of origin
#variabili da eliminare
column2drop = ['countryOfOrigin']
pp_class.drop(column2drop, axis=1, inplace=True)

{'Europe', 'Oceania', 'Africa', 'North America', 'Asia', 'South America'}


In [17]:
pp_class.head(15)

,rating,startYear,endYear,awardWins,numVotes,totalImages,totalVideos,totalCredits,criticReviewsTotal,awardNominationsExcludeWins,...,titleType_encoded,awardWins_bin,totalVideos_bin,awardNominations_bin,is_from_Europe,is_from_Oceania,is_from_Africa,is_from_North America,is_from_Asia,is_from_South America
0,8,1995,1995.0,0.0,779,1,0,21,1,0,...,0,0,0,0,0,0,0,1,0,0
1,6,1982,1982.0,0.0,11,1,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
2,6,1992,1992.0,0.0,38,1,0,24,0,0,...,2,0,0,0,1,0,0,0,0,0
3,6,2021,2021.0,0.0,15,22,0,56,0,0,...,3,0,0,0,0,0,0,1,0,0
4,8,2020,2020.0,1.0,15718,347,2,47,4,1,...,3,1,1,1,0,0,0,0,1,0
5,9,1995,1995.0,0.0,14,1,0,0,0,0,...,4,0,0,0,0,0,0,1,0,0
6,8,1995,1995.0,0.0,96,5,0,110,0,0,...,0,0,0,0,0,0,0,1,0,0
7,8,2022,2022.0,0.0,70,7,0,69,0,0,...,5,0,0,0,0,0,0,0,1,0
8,8,1996,1996.0,2.0,12,1,0,2,0,0,...,6,1,0,0,0,0,0,0,0,1
9,9,2022,2022.0,0.0,27,9,0,15,0,0,...,5,0,0,0,0,0,0,0,1,0


In [18]:
continent_distribution = pp_class[['is_from_South America', 'is_from_Oceania', 'is_from_Asia', 
                             'is_from_Africa', 'is_from_Europe', 'is_from_North America']].sum()

print(continent_distribution)

#somma totale > num record perchè alcuni multivalore

is_from_South America     405
is_from_Oceania           345
is_from_Asia             2536
is_from_Africa            154
is_from_Europe           5718
is_from_North America    7856
dtype: int64


### creazione array di valori

In [19]:
X_train = pp_class.values

### Splitting in validation

In [20]:

X_train2, X_val, y_train2, y_val = train_test_split(
     X_train, y, test_size=0.2, random_state=32
)

# X_train2 = training set --> 80% di X_train
# X_val = validation set --> 20% di X_train
# y_train2 e y_val sono i target corrispondenti (etichette) per i dati di addestramento e validazione

### Normalization

In [21]:

# standardsc = StandardScaler()

# # Fitta e trasforma il training set in un solo passaggio
# X_train_standardsc = standardsc.fit_transform(X_train2)
# X_val_standardsc = standardsc.fit_transform(X_val)

# # Trasforma il test set (senza rifittare!)
# #X_test_standardsc = standardsc.transform(X_test)

In [22]:
# minmax = MinMaxScaler()

# # Fitta e trasforma il training set in un solo passaggio
# X_train_minmax = minmax.fit_transform(X_train2)
# X_val_minmax = minmax.fit_transform(X_val)


# # Trasforma il test set (senza rifittare!)
# #X_test_minmax = minmax.transform(X_test)

# X_train_tr = X_train_minmax 
# X_val_tr = X_val_minmax 

## NAIVE BAYES